In [1]:
import os
import pandas as pd
from tqdm import tqdm as progessbar
from utils.constant import DIR_RAW

print("Loading Datasets...")
datasets = []

for filename in progessbar(os.listdir(DIR_RAW)):
    if filename.endswith(".csv"):
        file_path = os.path.join(DIR_RAW, filename)
        df = pd.read_csv(file_path, low_memory=False)
        datasets.append(df)

print("Datasets loaded.")

Loading Datasets...


100%|██████████| 6/6 [00:09<00:00,  1.55s/it]

Datasets loaded.


In [2]:
print("Cleaning Datasets...")

from utils import DatasetCleaner

cleaner = DatasetCleaner()
datasets = cleaner.clean(datasets)

print("Datasets cleaned.")

Cleaning Datasets...


Pulizia dei Dataset: 6it [00:01,  3.37it/s]

Datasets cleaned.


In [3]:
print("Feature Engineering...")

from utils import FeatureEngineering

fe = FeatureEngineering()
datasets = fe.apply(datasets)

print("Feature Engineering done.")

Feature Engineering...


Feature Engineering: 6it [00:00, 17.33it/s]


Feature Engineering done.


In [4]:
from utils import DatasetMerger

merger = DatasetMerger()
merger.process_and_save(datasets)

Merging datasets...
Datasets merged!
Dataset saved to ../data/cleaned/merged_dataset.parquet!
